In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

from src.model import UNet
from src.dataset import get_load_data
from src.loss_function import one_hot_encode
from torch.utils.data import DataLoader
from src.loss_function import energy_loss
from src.loss_function import dice_loss
from src.constants import weights
import torch.nn.functional as F
import torch

In [2]:
unet_model = UNet(num_classes=21)
train, test = get_load_data(root = "../../data", dataset = "VOCSegmentation", download = False)  
train_loader = DataLoader(train, batch_size=4)
imgs, smnts = next(iter(train_loader))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
imgs = imgs.to(device)
smnts = smnts.to(device)
weights = weights.to(device)
unet_model = unet_model.to(device)

In [3]:
import torch
import numpy as np

# img, smnt = train[0]
# img = img.reshape(1, 3, 572, 572)
# smnt = smnt.resize((388, 388))
# smnt = torch.tensor(np.asarray(smnt), dtype = torch.float32).reshape(1, 1, 388, 388)

pred = unet_model(imgs)
smnts_dice = smnts* 255
smnts_dice = torch.where(smnts_dice == 255, 0, smnts_dice)
e_loss = energy_loss(pred, smnts, weight = weights)
e_loss.backward()

In [4]:
pred = unet_model(imgs)

# have to do the few lines below because reasons
smnts_dice = smnts* 255
smnts_dice = torch.where(smnts_dice == 255, 0, smnts_dice)

d_loss = dice_loss(pred, smnts_dice, multiclass= True)
d_loss.backward()

<bound method Tensor.unique of tensor([[[[0.0900, 0.0615, 0.0500,  ..., 0.0260, 0.0432, 0.0094],
          [0.0636, 0.0595, 0.0769,  ..., 0.0744, 0.0514, 0.0043],
          [0.0309, 0.0429, 0.0324,  ..., 0.0795, 0.0293, 0.0091],
          ...,
          [0.0531, 0.0390, 0.0290,  ..., 0.0329, 0.0284, 0.0245],
          [0.0481, 0.0325, 0.0280,  ..., 0.0205, 0.0222, 0.0324],
          [0.0199, 0.0107, 0.0132,  ..., 0.0095, 0.0090, 0.0273]],

         [[0.0448, 0.0271, 0.0341,  ..., 0.0087, 0.0105, 0.0142],
          [0.0211, 0.0186, 0.0315,  ..., 0.0039, 0.0026, 0.0429],
          [0.0174, 0.0056, 0.0684,  ..., 0.0096, 0.0116, 0.0220],
          ...,
          [0.0168, 0.0382, 0.0433,  ..., 0.0688, 0.0420, 0.0392],
          [0.0173, 0.0316, 0.0277,  ..., 0.0555, 0.0269, 0.0307],
          [0.0274, 0.0295, 0.0186,  ..., 0.0268, 0.0230, 0.0195]],

         [[0.0259, 0.0691, 0.0407,  ..., 0.0407, 0.1090, 0.0377],
          [0.0277, 0.0771, 0.0317,  ..., 0.0722, 0.0152, 0.0288],
          [

In [5]:
d_loss

tensor(0.9779, device='cuda:0', grad_fn=<RsubBackward1>)